In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.keras import layers, models, regularizers

In [3]:
mypower = layers.Lambda(lambda x: tf.math.pow(x, 2))

In [6]:
mypower(tf.range(5))

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 0,  1,  4,  9, 16], dtype=int32)>

In [19]:
class Linear(layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(Linear, self).__init__(**kwargs)
        self.units = units
    def build(self, input_shape):
        self.w = self.add_weight('w', shape=(input_shape[-1], self.units),
                                initializer='random_normal',
                                 trainable=True
                                )
        self.b = self.add_weight('b', shape=(self.units, ),
                                initializer='random_normal',
                                 trainable=True
                                )
        super(Linear, self).build(input_shape)
        
    @tf.function
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        config = super(Linear, self).get_config()
        config.updatete({'units': self.units})
        return config
    
    
    

In [21]:
linear = Linear(units = 8)

In [22]:
print(linear.built)

False


In [23]:
linear.build(input_shape=(None, 16))

In [24]:
print (linear.built)

True


In [25]:
linear.compute_output_shape(input_shape=(None, 16))

TensorShape([None, 8])

In [26]:
linear = Linear(units = 16)

In [27]:
print(linear.built)

False


In [28]:
linear(tf.random.uniform((100, 64)))

<tf.Tensor: shape=(100, 16), dtype=float32, numpy=
array([[ 0.08764458,  0.02364155, -0.14989907, ..., -0.00860135,
         0.01703249, -0.20315196],
       [ 0.35334265,  0.05173421, -0.10992979, ..., -0.0531763 ,
        -0.00657489, -0.13177675],
       [ 0.20808983,  0.17651498, -0.22042191, ..., -0.00172485,
        -0.08502083,  0.05969475],
       ...,
       [ 0.25195563,  0.06156455, -0.20399901, ...,  0.0612418 ,
        -0.11148908, -0.07849316],
       [ 0.4470333 ,  0.13372988, -0.1253936 , ..., -0.02171433,
        -0.14525929, -0.1123251 ],
       [ 0.11720109,  0.23698862, -0.30167475, ...,  0.18674105,
        -0.05346302, -0.21012545]], dtype=float32)>

In [29]:
linear.built

True

In [30]:
config = linear.get_config()
config

{'name': 'linear_3', 'trainable': {}, 'dtype': 'float32', 'units': 16}

In [31]:
tf.keras.backend.clear_session()

In [35]:
class Linear(layers.Layer):
    def __init__(self, units=32, **kwargs):
        super(Linear, self).__init__(**kwargs)
        self.units = units
    
    #build方法一般定义Layer需要被训练的参数。    
    def build(self, input_shape): 
        self.w = self.add_weight("w",shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True) #注意必须要有参数名称"w",否则会报错
        self.b = self.add_weight("b",shape=(self.units,),
                                 initializer='random_normal',
                                 trainable=True)
        super(Linear,self).build(input_shape) # 相当于设置self.built = True

    #call方法一般定义正向传播运算逻辑，__call__方法调用了它。  
    @tf.function
    def call(self, inputs): 
        return tf.matmul(inputs, self.w) + self.b
    
    #如果要让自定义的Layer通过Functional API 组合成模型时可以被保存成h5模型，需要自定义get_config方法。
    def get_config(self):  
        config = super(Linear, self).get_config()
        config.update({'units': self.units})
        return config

In [36]:
model = models.Sequential()
#注意该处的input_shape会被模型加工，无需使用None代表样本数量维
model.add(Linear(units = 1,input_shape = (2,)))  
print("model.input_shape: ",model.input_shape)
print("model.output_shape: ",model.output_shape)
model.summary()

model.input_shape:  (None, 2)
model.output_shape:  (None, 1)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
linear_1 (Linear)            (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.compile(optimizer='sgd', loss='mse', metrics=['mae'])

In [38]:
model.predict(tf.constant([[3., 2.], [4., 5.]]))

array([[0.04112355],
       [0.20769626]], dtype=float32)

In [39]:
model.save('./models/linear_models_1', save_format='h5')

In [42]:
model_load_keras = tf.keras.models.load_model('./models/linear_models_1', 
                                              custom_objects={'Linear':Linear})

In [43]:
model_load_keras.predict(tf.constant([[3., 2.], [4., 5.]]))

array([[0.04112355],
       [0.20769626]], dtype=float32)

In [44]:
# 保存成 tf模型
model.save("./models/linear_model_2",save_format = "tf")
model_loaded_tf = tf.keras.models.load_model("./models/linear_model_2")
print(model_loaded_tf.predict(tf.constant([[3.0,2.0],[4.0,5.0]])))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./models/linear_model_2/assets
[[0.04112355]
 [0.20769626]]
